# Convert Terms for sDIC Decomposition

See Evernote journal from LANL summer, June 1st. Just conveting all the post-processed
means and residuals for each EBU for the terms relevant to the sDIC decomposition. This ensures that they are all mol/m3/yr.



In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def load_files(ebu, var):
    filepath = ('/glade/p/work/rbrady/EBUS_BGC_Variability/' + var + '/' + ebu + 
               '/filtered_output/')
    ds1 = xr.open_dataset(filepath + ebu.lower() + '-' + var + 
                          '-forced-signal-AW-chavez-800km.nc')
    ds2 = xr.open_dataset(filepath + ebu.lower() + '-' + var +
                          '-forced-signal-chavez-800km.nc')
    ds3 = xr.open_dataset(filepath + ebu.lower() + '-' + var +
                          '-residuals-AW-chavez-800km.nc')
    ds4 = xr.open_dataset(filepath + ebu.lower() + '-' + var +
                          '-residuals-chavez-800km.nc')
    return ds1, ds2, ds3, ds4

In [3]:
def convert_and_save(ds1, ds2, ds3, ds4, conversion, ebu, var):
    ds1 = ds1 * conversion
    ds1[var + '_AW'].attrs['units'] = 'mol/m2/yr'
    if var == 'Jint_100m_DIC':
        ds1[var + '_AW'].attrs['description'] = ("Source/sink biological impact on DIC" +
                                                 "integrated over 100m.")
    elif var == 'FG_CO2':
        ds1[var + '_AW'].attrs['description'] = "Native CO2 flux. Positive is flux out of ocean."
    
    ds2 = ds2 * conversion
    ds2[var].attrs['units'] = 'mol/m2/yr'
    if var == 'Jint_100m_DIC':
        ds2[var].attrs['description'] = ("Source/sink biological impact on DIC" +
                                                 "integrated over 100m.")
    elif var == 'FG_CO2':
        ds2[var].attrs['description'] = "Native CO2 flux. Positive is flux out of ocean."

    ds3 = ds3 * conversion
    ds3[var + '_AW'].attrs['units'] = 'mol/m2/yr'
    if var == 'Jint_100m_DIC':
        ds3[var + '_AW'].attrs['description'] = ("Source/sink biological impact on DIC" +
                                                 "integrated over 100m.")
    elif var == 'FG_CO2':
        ds3[var + '_AW'].attrs['description'] = "Native CO2 flux. Positive is flux out of ocean."
        
    ds4 = ds4 * conversion
    ds4[var].attrs['units'] = 'mol/m2/yr'
    if var == 'Jint_100m_DIC':
        ds4[var].attrs['description'] = ("Source/sink biological impact on DIC" +
                                                 "integrated over 100m.")
    elif var == 'FG_CO2':
        ds4[var].attrs['description'] = "Native CO2 flux. Positive is flux out of ocean."
        
    outdir = ('/glade/p/work/rbrady/EBUS_BGC_Variability/sDIC_decomposition/' + 
              var + '/' + ebu + '/')
    # Save files
    ds1.to_netcdf(outdir + ebu.lower() + '-' + var + '-forced-signal-AW-chavez-800km.nc')
    ds2.to_netcdf(outdir + ebu.lower() + '-' + var + '-forced-signal-chavez-800km.nc')
    ds3.to_netcdf(outdir + ebu.lower() + '-' + var + '-residuals-AW-chavez-800km.nc')
    ds4.to_netcdf(outdir + ebu.lower() + '-' + var + '-residuals-chavez-800km.nc')

# Convert Terms

In [16]:
EBU = 'BenCS'
VAR = 'FG_CO2'

In [17]:
conversions = {'sDIC_int100m_tendency': (12/10**3),
               'Jint_100m_DIC': (60*60*24*365) / (10**5),
               'FG_CO2': 1}

In [18]:
# Tendency conversion
ds1, ds2, ds3, ds4 = load_files(EBU, VAR)
convert_and_save(ds1, ds2, ds3, ds4, conversions[VAR], EBU, VAR)

/glade/u/home/rbrady/miniconda3/envs/py36/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


# Make circulation residual

Ref: Lovenduski et al. 2007

$$
    \frac{d(sDIC^{\prime})}{dt} = J^{\prime}_{circ} + J^{\prime}_{ex} + J^{\prime}_{bio}
$$

Where $J^{\prime}_{ex}$ is positive in this case due to flipping the sign of FG_CO2. $ J^{\prime}_{bio}$ is also positive given the definition of it in CESM.

In [20]:
def load_updated_files(ebu, var):
    filepath = ('/glade/p/work/rbrady/EBUS_BGC_Variability/sDIC_decomposition/' 
                + var + '/' + ebu + '/')
    ds1 = xr.open_dataarray(filepath + ebu.lower() + '-' + var + 
                          '-forced-signal-AW-chavez-800km.nc')
    ds2 = xr.open_dataarray(filepath + ebu.lower() + '-' + var +
                          '-forced-signal-chavez-800km.nc')
    ds3 = xr.open_dataarray(filepath + ebu.lower() + '-' + var +
                          '-residuals-AW-chavez-800km.nc')
    ds4 = xr.open_dataarray(filepath + ebu.lower() + '-' + var +
                          '-residuals-chavez-800km.nc')
    return ds1, ds2, ds3, ds4

In [21]:
def determine_circ(EBU):
    # Gas flux
    co2_1, co2_2, co2_3, co2_4 = load_updated_files(EBU, 'FG_CO2')
    co2_3 = co2_3.isel(ensemble=slice(1,34))
    co2_4 = co2_4.isel(ensemble=slice(1,34))    
    # Bio activity
    bio_1, bio_2, bio_3, bio_4 = load_updated_files(EBU, 'Jint_100m_DIC')
    bio_3 = bio_3.isel(ensemble=slice(1,34))
    bio_4 = bio_4.isel(ensemble=slice(1,34))
    # sDIC tendency
    tend_1, tend_2, tend_3, tend_4 = load_updated_files(EBU, 
                                                        'sDIC_int100m_tendency')
    # Derive circulation
    circ_1 = tend_1 - co2_1 - bio_1
    circ_1.name ='Jint_100m_circ'
    circ_1.attrs['units'] = 'mol/m2/yr'
    circ_1.attrs['description'] = 'Residual computed by subtracting J_ex and J_bio from the reconstructed tendency term.'

    circ_2 = tend_2 - co2_2 - bio_2
    circ_2.name ='Jint_100m_circ'
    circ_2.attrs['units'] = 'mol/m2/yr'
    circ_2.attrs['description'] = 'Residual computed by subtracting J_ex and J_bio from the reconstructed tendency term.'
    
    circ_3 = tend_3 - co2_3 - bio_3
    circ_3.name ='Jint_100m_circ'
    circ_3.attrs['units'] = 'mol/m2/yr'
    circ_3.attrs['description'] = 'Residual computed by subtracting J_ex and J_bio from the reconstructed tendency term.'
    
    circ_4 = tend_4 - co2_4 - bio_4
    circ_4.name ='Jint_100m_circ'
    circ_4.attrs['units'] = 'mol/m2/yr'
    circ_4.attrs['description'] = 'Residual computed by subtracting J_ex and J_bio from the reconstructed tendency term.'
    
    # Save out
    outdir = ('/glade/p/work/rbrady/EBUS_BGC_Variability/sDIC_decomposition/Jint_100m_circ/' + EBU + '/')
    
    print("saving out 1...")
    circ_1.to_dataset().to_netcdf(outdir + EBU.lower() + '-forced-signal-AW-chavez-800km.nc')
    print("saving out 2...")
    circ_2.to_dataset().to_netcdf(outdir + EBU.lower() + '-forced-signal-chavez-800km.nc')
    print("saving out 3...")
    circ_3.to_dataset().to_netcdf(outdir + EBU.lower() + '-residuals-AW-chavez-800km.nc')
    print("saving out 4...")
    circ_4.to_dataset().to_netcdf(outdir + EBU.lower() + '-residuals-chavez-800km.nc')

In [24]:
determine_circ('BenCS')


saving out 1...
saving out 2...
saving out 3...
saving out 4...


/glade/u/home/rbrady/miniconda3/envs/py36/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")
